In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

# Quantum circuit parameters
num_qubits = 4
num_layers = 3
num_classes = 28  # Adjust based on your dataset
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev, interface='tf')
def quantum_circuit(inputs, weights):
    qml.templates.AmplitudeEmbedding(inputs, wires=range(num_qubits), normalize=True, pad_with=0.0)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        self.q_weights = self.add_weight(
            name='q_weights', shape=(num_layers, num_qubits, 3), initializer='random_normal'
        )

    def call(self, inputs):
        outputs = tf.vectorized_map(lambda x: quantum_circuit(x, self.q_weights), inputs)
        return tf.cast(outputs, tf.float32)


2024-05-07 15:04:02.473370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-07 15:04:02.949465: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
# Create the model using the functional API
def create_model(input_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(2 ** num_qubits)(x)  # Ensure 16 output features
    x = tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 2 ** num_qubits)))(x)
    x = QuantumLayer()(x)
    x = tf.keras.layers.Flatten()(x)  # Flatten quantum outputs
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create and compile the model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Cause: Unable to locate the source code of <function _gcd_import at 0x7f4c14509430>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7f4c14509430>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @t

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (4, None)

In [ ]:
# Data Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    'data/train', target_size=(32, 32), batch_size=32, class_mode='categorical'
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    'data/test', target_size=(32, 32), batch_size=32, class_mode='categorical'
)

print(f"Found {train_generator.samples} images in {train_generator.num_classes} classes for training.")
print(f"Found {test_generator.samples} images in {test_generator.num_classes} classes for testing.")


Found 2342 images belonging to 28 classes.
Found 236 images belonging to 27 classes.
Found 2342 images in 28 classes for training.
Found 236 images in 27 classes for testing.


In [ ]:
# Train the model
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/raag/sdp/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/raag/sdp/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/raag/sdp/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/raag/sdp/.venv/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_fo